# AgriSmart

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
import joblib

In [ ]:
DATASET_DIR = "./PlantVillage/" 

# Model parameters
IMG_HEIGHT = 128
IMG_WIDTH = 128
BATCH_SIZE = 32

In [ ]:
# Prepare data using ImageDataGenerator with augmentation
datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
print("\n--- Loading Data ---")
train_data = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training"
)

Found 33027 images belonging to 16 classes.


In [ ]:
val_data = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)

Found 8249 images belonging to 16 classes.


In [ ]:
# Verify the classes
class_names = list(train_data.class_indices.keys())
print("\nClasses found:", class_names)
NUM_CLASSES = len(class_names)

c:\Users\Akshat\PROJECTS\AgriSmart\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:

# ## 3. Build and Compile the CNN Model
print("\n--- Building Model ---")
model = Sequential([
    keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    keras.layers.Conv2D(32, (3, 3), activation="relu"),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(64, (3, 3), activation="relu"),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(128, (3, 3), activation="relu"),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(NUM_CLASSES, activation="softmax")
])

In [ ]:
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)
model.summary()

In [ ]:
# ## 4. Train the Model
print("\n--- Starting Model Training ---")
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10  # You can increase this later for better accuracy
)
print("\nTraining finished successfully!")

✅ Model saved as disease_model.h5

✅ Disease labels found: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'PlantVillage', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']
✅ Encoder saved successfully as disease_label_encoder.pkl


In [ ]:

# ## 5. Save the Final Model and Encoder
print("\n--- Saving Artifacts ---")

# Save the trained model
# Using the .keras format is recommended over .h5
model.save("disease_model.keras")
print("✅ Model saved as disease_model.keras")

In [ ]:

# Create and save the label encoder
disease_label_encoder = LabelEncoder()
disease_label_encoder.fit(class_names)
joblib.dump(disease_label_encoder, 'disease_label_encoder.pkl')
print("✅ Label encoder saved as disease_label_encoder.pkl")